In [1]:
import os
import json

from pprint import pprint

from langchain import LLMChain, PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, AgentType, initialize_agent
from langchain.llms import vertexai
from langchain.tools.base import BaseTool
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)


from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform

from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, Type

In [2]:
# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# authenticate 
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456
aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")


## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [5]:
from tools import GetOrderIdTool, OrderTool, GetDetailedMenuTool, FindItemIdTool

tools = [GetOrderIdTool(), GetDetailedMenuTool(), FindItemIdTool(), OrderTool()]

In [6]:
agent = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [8]:
agent.run("Could I get a large coffee?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "find_item_id_tool",
  "action_input": {
    "name": "coffee large"
  }
}
```

Observation: Error: cannot find item in menu.
Thought: Action:
```
{
  "action": "get_detailed_menu_tool",
  "action_input": {}
}
```


Observation: ['Small, Blueberry Muffin', 'Small, Banana Chocolate Chip Muffin', 'Small, Radish Muffin', 'Medium, Blueberry Muffin', 'Medium, Banana Chocolate Chip Muffin', 'Medium, Radish Muffin', 'Large, Blueberry Muffin', 'Large, Banana Chocolate Chip Muffin', 'Large, Radish Muffin', 'Small Coffee', 'Medium Coffee', 'Large Coffee', 'Blueberry Danish', 'Banana Chocolate Chip Danish', 'Radish Danish']
Thought: Action:
```
{
  "action": "find_item_id_tool",
  "action_input": {
    "name": "coffee large"
  }
}
```


Observation: Error: cannot find item in menu.
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "Sorry, we don't have large coffee. Would you like to order something else?"
}
```


"Sorry, we don't have large coffee. Would you like to order something else?"

## Chirp
TODO: setup google chirp just for a simple demo